<a href="https://colab.research.google.com/github/nhanlunlk2k/APP-project/blob/main/app_group7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import numba
import copy

In [ ]:
def find_min_max_2D(M):
    min = M[0][0]
    max = M[0][0]
    for i in range(len(M)):
        for j in range(len(M[i])):
            if M[i][j] < min:
                min = M[i][j]
            elif M[i][j] > max:
                max = M[i][j]
    return min,max

def find_mean_2D(M):
    sum = 0
    for i in range(len(M)):
        for j in range(len(M[i])):
            sum += M[i][j]
    return sum/(len(M)*len(M[0]))

def find_median_2D(M):
    tmp = copy.copy(M[0])
    for i in range(1,len(M)):
        tmp.extend(M[i])
    tmp.sort()
    if len(tmp)%2 == 0:
        return (tmp[len(tmp)//2] + tmp[len(tmp)//2 - 1])/2
    else:
        return tmp[len(tmp)//2]

def pad_2D(M,padding_size,mode):
    num_cols = len(M[0]) + padding_size[0][0] + padding_size[0][1]
    #num_rows = len(M) + padding_size[1][0] + padding_size[1][1]
    match mode:
        case 'min':
            cons = find_min_max_2D(M)[0]
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            line = [cons]*num_cols
            for j in range(padding_size[1][0]):
                M.insert(0,line)
            for j in range(padding_size[1][1]):
                M.append(line)
            return M
        case 'max':
            cons = find_min_max_2D(M)[1]
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            line = [cons]*num_cols
            for j in range(padding_size[1][0]):
                M.insert(0,line)
            for j in range(padding_size[1][1]):
                M.append(line)
            return M
        case 'mean':
            cons = find_mean_2D(M)
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            for j in range(padding_size[1][0]):
                M.insert(0,[cons]*num_cols)
            for j in range(padding_size[1][1]):
                M.append([cons]*num_cols)
            return M
        case 'zero':
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,0)
                for j in range(padding_size[0][1]):
                    M[i].append(0)
            for j in range(padding_size[1][0]):
                M.insert(0,[0]*num_cols)
            for j in range(padding_size[1][1]):
                M.append([0]*num_cols)
            return M
        case 'median':
            cons = find_median_2D(M)
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,cons)
                for j in range(padding_size[0][1]):
                    M[i].append(cons)
            for j in range(padding_size[1][0]):
                M.insert(0,[cons]*num_cols)
            for j in range(padding_size[1][1]):
                M.append([cons]*num_cols)
            return M
        case 'edge':
            for i in range(len(M)):
                for j in range(padding_size[0][0]):
                    M[i].insert(0,M[i][0])
        #print(M)
                for j in range(padding_size[0][1]):
                    M[i].append(M[i][-1])
        #print(M)
            M = [M[0]]*padding_size[1][0] + M + [M[-1]]*padding_size[1][1]
    #print(M)
            return M
        case _:
            raise ValueError('Invalid mode')
    pass

In [ ]:
def num_dim(A):
    if type(A) == list:
        return 1 + num_dim(A[0])
    else:
        return 0

def array_shape(A):
    shape = []
    while type(A) == list:
        shape.append(len(A))
        A = A[0]
    return shape

#convolution using Python
def convolution_py(F,M,padding=None): #F is a filters (n filters x 2d array), M is a volume (3d array)
    F_size = array_shape(F)
    up_padding = F_size[1]//2
    down_padding = F_size[1] - F_size[1]//2 - 1
    left_padding = F_size[2]//2
    right_padding = F_size[2] - F_size[2]//2 - 1
    pad_size = ((left_padding,right_padding),(up_padding,down_padding))
    res = []
    if padding != None:
        for i in range(len(M)):
            M[i] = pad_2D(M[i],pad_size,padding)
        res = [[[0]*len(M[0][0])]*len(M[0])]*F_size[0]
    else:
        res = [[[0]*(len(M[0][0]) - F_size[2] + 1)]*(len(M[0]) - F_size[1] + 1)]*F_size[0]
    M_size = array_shape(M)
    tmp = [[0]*M_size[1]]*M_size[2]
    for i in range(M_size[1]):
        for j in range(M_size[2]):
            for k in range(M_size[0]):
                tmp[i][j] += M[k][i][j]
    #for i in range(len(res)):
    #    for j in range(len(res[i])):
    #        for i_f in range(len(F)):
    #            for j_f in range(len(F[i_f])):
    #                res[i][j] += tmp[i+i_f][j+j_f]*F[i_f][j_f]
    for k in range(len(res)):
        for i in range(len(res[i])):
            for j in range(len(res[i][j])):
                for f in range(len(F)):
                    for i_f in range(len(F[f])):
                        for j_f in range(len(F[f][i_f])):
                            res[k][i][j] += tmp[i+i_f][j+j_f]*F[f][i_f][j_f]
    return res

def pooling_py(M,pool_size,type_,stride): #M has n matrices
    res = [[]]*(len(M) // stride[0])
    M_size = array_shape(M)
    for k in range(M_size[0]):
        for i in range(0,M_size[1],stride[0]):
            line = []
            for j in range(0,M_size[2],stride[1]):
                tmp = []
                for i_p in range(pool_size[0]):
                    for j_p in range(pool_size[1]):
                        if (i+i_p < len(M)) and (j+j_p < len(M[0])):
                            tmp.append(M[i+i_p][j+j_p])
                        else:
                            pass
                match type_:
                    case 'max':
                        line.append(max(tmp))
                    case 'min':
                        line.append(min(tmp))
                    case 'mean':
                        line.append(sum(tmp)/len(tmp))
            res[k].append(line)
    return res

def unpooling_py(M,pool_size,stride):
    pass

def deconvolution_py(F,M):
    pass

In [ ]:
#Convolution using Numpy
def convolution_np(F,M,padding=None): #F is filters (n filters x 2d array), M is a volume (3d array)
    up_padding = F.shape[1]//2
    down_padding = F.shape[1] - F.shape[1]//2 - 1
    left_padding = F.shape[2]//2
    right_padding = F.shape[2] - F.shape[2]//2 - 1
    pad_size = ((left_padding,right_padding),(up_padding,down_padding))
    res = []
    if padding != None:
        match padding:
            case 'min':
                M = np.pad(M,pad_size,mode='minimum')
            case 'max':
                M = np.pad(M,pad_size,mode='maximum')
            case 'mean':
                M = np.pad(M,pad_size,mode='mean')
            case 'zero':
                M = np.pad(M,pad_size,mode='constant',constant_values=0)
            case 'median':
                M = np.pad(M,pad_size,mode='median')
            case 'edge':
                M = np.pad(M,pad_size,mode='edge')
            case _:
                raise ValueError('Invalid padding mode')
        res = np.zeros((F.shape[0],M.shape[1],M.shape[2]))
    else:
        res = np.zeros((F.shape[0],M.shape[1] - F.shape[1] + 1,M.shape[2] - F.shape[2] + 1))
    tmp = M.sum(axis=0)
    for k in range(res.shape[0]):
        for i in range(res.shape[1]):
            for j in range(res.shape[2]):
                res[k][i][j] += (F * tmp[i:i + F.shape[1] + 1,j:j + F.shape[2] + 1]).sum(axis=0)[k]
    return res

def pooling_np(M,pool_size,type_,stride):
    res = np.zeros((M.shape[0],M.shape[1]//stride[0],M.shape[2]//stride[1]))
    for k in range(0,M.shape[0]):
        for i in range(0,M.shape[1],stride[0]):
            for j in range(0,M.shape[2],stride[1]):
                tmp = M[:,i:i + pool_size[0],j:j + pool_size[1]]
            match type_:
                case 'max':
                    res[k][i][j] = np.max(tmp,axis=0)[k]
                case 'min':
                    res[k][i][j] = np.min(tmp,axis=0)[k]
                case 'mean':
                    res[k][i][j] = np.mean(tmp,asix=0)[k]
    return res

def unpooling_np(M,pool_size,stride):
    pass

def deconvolution_np(F,M):
    pass

In [ ]:
a = [[6,1,8],[3,2,5],[9,5,7],[4,3,6]]
#tmp = a[0]
#for i in range(1,len(a)):
#    tmp += a[i]
#tmp.sort()
#print(tmp)
#tmp.sort()
#tmp
array_shape(a)

[4, 3]

In [ ]:
a = np.array(a)
#a.flatten()

In [ ]:
b = np.array([[[1,3,5,7],[2,4,6,8],[2,6,7,1],[9,4,10,3]],[[2,3,9,6],[1,7,6,5],[10,13,9,6],[2,4,5,10]]])


array([[ 2,  3,  9,  6],
       [ 1,  7,  6,  5],
       [10, 13,  9,  6],
       [ 2,  4,  5, 10]])